Example of looping through all comments, and storing info in a dataframe

In [1]:
#Grab packages

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Connect to webpage

options = webdriver.ChromeOptions()
#options.add_argument('--headless')


options.binary_location = "/usr/bin/google-chrome"
chrome_driver_binary = "/usr/local/bin/chromedriver"
#options.binary_location = "C:/Program Files (x86)/Google/Chrome/Application/chrome.exe"
#chrome_driver_binary = "C:/Users/Alek/chromedriver"
driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)



/home/ravisolter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [3]:
# Logging in
driver.get("https://auth.voxmedia.com/login?return_to=https://www.blackwhitereadallover.com/")

element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "p-text-input"))
    )

username = driver.find_element_by_xpath('//*[@id="login[username]"]')
username.clear()
username.send_keys("rsolter")


password = driver.find_element_by_xpath('//*[@id="login[password]"]')
password.clear()
password.send_keys("pass123!")


button = driver.find_element_by_xpath('//*[@id="auth"]/div/form/fieldset[3]/input')
button.click()

In [4]:
# Directing to one example article
element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "c-global-header__logo-large"))
    )




In [5]:

### Insert loop here to visit relevant URLs

driver.get("https://www.blackwhitereadallover.com/2020/4/3/21206623/miralem-pjanic-marco-verratti-juventus-paris-saint-germain-2020-serie-a-summer-transfer-rumors#comments")

## Grab article title, author

element = WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "c-comments__list"))
    )

#Find number of comments

n_comments = driver.find_element_by_xpath('//*[@id="comments"]/div[1]/span/span')
n_comments = int(n_comments.text)


In [49]:
#Loop through all comments, taking title, content, and meta data (recs, author, etc)

comments_df = pd.DataFrame()

for i in range(1,n_comments+1):
    
    xpath = '//*[@id="comments"]/div[2]/div['+str(i)+']'
     
    title = driver.find_element_by_xpath(xpath + '/div[1]')
    body = driver.find_element_by_xpath(xpath + '/div[2]')
    meta = driver.find_element_by_xpath(xpath + '/div[3]')
    
    
    # Grabbing author, comment recommendations
    #author_xpath = '/html/body/div[2]/main/article/div[5]/div[1]/section/div[2]/div['+str(i)+']/div[3]/span[1]/a[1]'
    #author = driver.find_element_by_xpath(author_xpath)
    
    # Recommendations number specifically doesn't work when there are none
    #recs_xpath = '/html/body/div[2]/main/article/div[5]/div[1]/section/div[2]/div['+str(i)+']/div[3]/span[2]/button[2]/span[2]'
        
    # Also grabbing the entire comments_action errors out on 10th iteration which has a div[4] instead of div[3]
    #recs_xpath = '/html/body/div[2]/main/article/div[5]/div[1]/section/div[2]/div['+str(i)+']/div[3]/span[2]'
    #recs = driver.find_element_by_xpath(recs_xpath)

    comment = pd.DataFrame([title.text,body.text,meta.text])
    comments_df = pd.concat([comments_df,comment.T])
    
    ;
   

1 #finoallafine
2 #finoallafine
3 #finoallafine
4 #finoallafine
5 #finoallafine
6 #finoallafine
7 #finoallafine
8 #finoallafine
9 #finoallafine
10 #finoallafine
11 #finoallafine
12 #finoallafine
13 #finoallafine
14 #finoallafine
15 #finoallafine
16 #finoallafine
17 #finoallafine
18 #finoallafine
19 #finoallafine
20 #finoallafine
21 #finoallafine
22 #finoallafine
23 #finoallafine
24 #finoallafine
25 #finoallafine
26 #finoallafine
27 #finoallafine
28 #finoallafine
29 #finoallafine
30 #finoallafine
31 #finoallafine
32 #finoallafine
33 #finoallafine
34 #finoallafine
35 #finoallafine
36 #finoallafine
37 #finoallafine
38 #finoallafine
39 #finoallafine
40 #finoallafine
41 #finoallafine
42 #finoallafine


In [54]:
## The loop below works for handling NAs, but it's returning recs from only the first level of comments

from selenium.common.exceptions import NoSuchElementException

# Searching on recommend element specifically (handing NAs)

recs_df = pd.DataFrame()

for i in range(1,n_comments+1):
    
    try:
        recs_xpath = '/html/body/div[2]/main/article/div[5]/div[1]/section/div[2]/div['+str(i)+']/div[3]/span[2]/button[2]/span[2]'
        recs = driver.find_element_by_xpath(recs_xpath)
        if recs.is_displayed():
            recN = recs.text
            print(recN)
    except NoSuchElementException:        
        print("...")    

    #print(i,recs.text)
    ;
    
    
    

...
(8)
...
(4)
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
(1)
...
...
...
...
...
...


In [36]:
## Searching for comments by name returns empty lists

#recs_list = driver.find_elements_by_name('c-comments__actions')
recs_list = driver.find_elements_by_name('icon-rec c-comments__rec-link ') 
#recs_list = driver.find_elements_by_name('c-comments__post-actions u-clearfix')

type(recs_list)
print(recs_list)




[]


In [53]:
comments_df.columns = ['Title','Body','Meta']

comments_df.head()


comments_df.to_csv('comments.csv', index=False)




In [40]:
driver.quit()